# MC Code RAG Workshop


### Dependencies


In [ ]:
# %pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4 python-dotenv

In [1]:
import os
from dotenv import load_dotenv,find_dotenv

# Use this line of code if you have a local .env file
load_dotenv(find_dotenv()) 

# LangSmith
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.environ["LANGCHAIN_API_KEY"] #getpass.getpass()
 
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader, TextLoader, PyPDFLoader, PyPDFium2Loader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document

# Txt loader
# loader = TextLoader("./docs/WMX3UserManual_a.txt")
loader = TextLoader("./docs/WMX3API_MCEval_Samplecodes.txt")
docs = loader.load()
# docs[0].page_content[:100000]
len(docs)

# # Text chunk 
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
# splits = text_splitter.split_documents(docs)

#Sample code chunk with dedicated separators
separators = ['``']  # Adjust based on actual document structure, `` is the end of each code snippet.
# separators = ['}']  # Adjust based on actual document structure
text_splitter = RecursiveCharacterTextSplitter(separators=separators, keep_separator=True, chunk_size=1000, chunk_overlap=200, add_start_index=True)
splits = text_splitter.split_documents(docs)


In [4]:
splits[0]

Document(page_content='# Write python code to Initialize WMX3, do {user_logic}, and close WMX3.\n# This python code is to initialized WMX3(includes CreateDevice, SetDeviceName, StartCommunication, ImportAndSetAll, ClearAmpAlarm, SetServoOn, StartHome), and do {user_logic}, and close WMX3(includes SetServoOn, StopCommunication and CloseDevice). \n#WMX3 python library\n# Axes = []\n\nfrom WMX3ApiPython import *\nfrom time import *\n\nINFINITE = int(0xFFFFFFFF)\n\ndef main():\n    Wmx3Lib = WMX3Api()\n    CmStatus = CoreMotionStatus()\n    Wmx3Lib_cm = CoreMotion(Wmx3Lib)\n    print(\'Program begin.\')\n    sleep(0.1)\n\n    # Create devices. \n    ret = Wmx3Lib.CreateDevice(\'C:\\\\Program Files\\\\SoftServo\\\\WMX3\', DeviceType.DeviceTypeNormal, INFINITE)\n    if ret!=0:\n        print(\'CreateDevice error code is \' + str(ret) + \': \' + Wmx3Lib.ErrorToString(ret))\n        return\n    \n    # Set Device Name.\n    Wmx3Lib.SetDeviceName(\'WMX3initTest\')\n\n    # Start Communication.\

In [5]:


embedding_model=OpenAIEmbeddings(model="text-embedding-3-large")   #text-embedding-3-large   #text-embedding-ada-002    #text-embedding-3-small

# If txt vectorstore exists
# if os.path.exists("Vectorstore/chromadb"):
        # vectorstore = Chroma(
                #     embedding_function=embedding_model,
                #     persist_directory="Vectorstore/chromadb",
                #     ) 
# else:
        # Load from chunks and save to disk
        # vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model, persist_directory="Vectorstore/chromadb") 

# If pdf vectorstore exists
vectorstore_path = "Vectorstore/chromadb-MCCoder"
 # Load from chunks and save to disk
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model,  persist_directory=vectorstore_path) 


In [6]:

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

retrieved_docs = retriever.invoke("Execute path interpolation with look ahead of Axis 0, 1 and 2 ")
retrieved_docs

[Document(page_content='Overview \n\nPath interpolation with look ahead is a more advanced version of path interpolation with \nthe following features:', metadata={'source': './docs/WMX3UserManual_a.txt', 'start_index': 862601}),
 Document(page_content="``\n\n\n\n# Write python code to Execute path interpolation with look ahead of Axis 0 and Axis 1 with velocity 1000, consisting of four linear interpolations: (100,0),(100,100),(0,100),(0,0).\n    # Axes = [0, 1]\n\n    Wmx3Lib_adv = AdvancedMotion(Wmx3Lib)\n\n    path = AdvMotion_PathIntplLookaheadCommand()\n    ret = Wmx3Lib_adv.advMotion.FreePathIntplLookaheadBuffer(0)\n    # Create the path interpolation with look ahead buffer\n    ret = Wmx3Lib_adv.advMotion.CreatePathIntplLookaheadBuffer(0, 1000)\n    if ret != 0:\n        print('CreatePathIntplLookaheadBuffer error code is ' + str(ret) + ': ' + Wmx3Lib_adv.ErrorToString(ret))\n        return\n\n    # Configure the path interpolation with look ahead channel\n    conf = AdvMotion_P